# Illustration of the doubling-distance code
This notebook aims at illustrating how a doubling distance code like 

![image.png](./images/doubling_distance.png)

can be implemented using the template approach.

## 1. Define the building blocks

The first step to create a template is to define the base blocks (basically, every arrangement of plaquettes enclosed in a green shape in the image above) that will be used to build it.

The `tqec` package already implement some of the most widely used building blocks in the `tqec.templates.fixed` and `tqec.templates.scalable` modules.

**Important note**: In the following code cell, **ALL** plaquette indices are defined as `1`. This is **not** what we would like to generate for a QEC code, that would require to define different plaquettes. This simplification is done to avoid the clutter and complexity of plaquette numbering.

In [1]:
import typing as ty
import numpy
from tqec.templates.atomic.rectangle import (
    AlternatingRectangleTemplate,
    RawRectangleTemplate,
)
from tqec.templates.atomic.square import AlternatingSquareTemplate
from tqec.templates.base import TemplateWithIndices
from tqec.templates.scale import Dimension, LinearFunction

dim = Dimension(2, LinearFunction(2))
nsone = Dimension(1, LinearFunction(0, 1))
nstwo = Dimension(2, LinearFunction(0, 2))
left_white_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(nsone, dim), [1, 0]
)  # 2, 9, 19
right_white_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(nsone, dim), [0, 1]
)  # 4, 13, 23
top_white_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(dim, nsone), [1, 0]
)  # 8
top_black_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(dim, nsone), [0, 1]
)  # 1
bottom_black_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(dim, nsone), [1, 0]
)  # 24, 26
scalable_square = TemplateWithIndices(
    AlternatingSquareTemplate(dim), [1, 1]
)  # 3, 10, 20, 22
fixed_square = TemplateWithIndices(AlternatingSquareTemplate(nstwo), [1, 1])  # 16
horizontal_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(dim, nstwo), [1, 1]
)  # 6, 15, 17
vertical_scalable_rectangle = TemplateWithIndices(
    AlternatingRectangleTemplate(nstwo, dim), [1, 1]
)  # 11, 21
left_fixed_square = TemplateWithIndices(RawRectangleTemplate([[0]]), [1])
right_fixed_square = TemplateWithIndices(RawRectangleTemplate([[0]]), [1])
bottom_fixed_square = TemplateWithIndices(RawRectangleTemplate([[0]]), [1])
# Specific cases:
# - the corner numbered 7
corner = TemplateWithIndices(RawRectangleTemplate([[0]]), [1])

The only building block that is not already defined in the `tqec` package is the one numbered $12$. 

This building block is basically a regular `ScalableAlternatingSquare` with its top-right plaquette replaced by another plaquette. Following this observation, this basic block is implemented by subclassing the `ScalableAlternatingSquare` class and replacing the top-right plaquette by the special plaquette required.

In [2]:
# - the square numbered 12 that has a specific top-right plaquette
class SpecificAlternatingSquareTemplate(AlternatingSquareTemplate):
    def instantiate(
        self,
        plaquette_indices: ty.Sequence[int],
    ) -> numpy.ndarray:
        self._check_plaquette_number(plaquette_indices, 3)
        x_plaquette, z_plaquette, special_plaquette = plaquette_indices[:3]
        arr = super().instantiate([x_plaquette, z_plaquette])
        arr[0, -1] = special_plaquette
        return arr

    @property
    def expected_plaquettes_number(self) -> int:
        return 3


top_right_specific_square = TemplateWithIndices(
    SpecificAlternatingSquareTemplate(dim), [1, 1, 1]
)

Now that the building blocks are defined, we create a list of templates according to the numbering in the image at the beginning of this notebook. 

Note that this list contains duplicate instances, meaning that changing in-place one instance in the list *might* also change another item of the list. 
This is not an issue because:
- this list will only be used to initialise the `ComposedTemplate` instance and,
- the only operation in `ComposedTemplate` that modifies the underlying templates stored is when calling `ComposedTemplate.scale_to`, and the same scale is forwarded to all the stored templates.

Nevertheless, you might want to store copies to avoid any subtle reference issues if the `ComposedTemplate` implementation changes in the future. 

**Important note**: lists being 0-indexed in Python, the template indices in Python code will be shifted. As such, the template numbered `1` in the picture above will be the template indexed as `0` in the code below.

In [3]:
templates = [
    # 0
    top_black_scalable_rectangle,
    left_white_scalable_rectangle,
    scalable_square,
    right_white_scalable_rectangle,
    left_fixed_square,
    # 5
    horizontal_scalable_rectangle,
    corner,
    top_white_scalable_rectangle,
    left_white_scalable_rectangle,
    scalable_square,
    # 10
    vertical_scalable_rectangle,
    top_right_specific_square,
    right_white_scalable_rectangle,
    left_fixed_square,
    horizontal_scalable_rectangle,
    # 15
    fixed_square,
    horizontal_scalable_rectangle,
    right_fixed_square,
    left_white_scalable_rectangle,
    scalable_square,
    # 20
    vertical_scalable_rectangle,
    scalable_square,
    right_white_scalable_rectangle,
    bottom_black_scalable_rectangle,
    bottom_fixed_square,
    # 25
    bottom_black_scalable_rectangle,
]
assert len(templates) == 26

## 2. Define the relations

Now that the basic building blocks have been defined, we should encode the relationships between each block to connect them. There are two ways of specifying relationships.

### 2.1. Relative positioning

Building blocks can be positioned relatively to each other. The following code cell defines such relations in a list that contains 3-dimensional tuples containing:
1. The template that should be positioned.
2. The relative position of the template provided in 1.
3. The template used as an anchor to position the template provided in 1.

This order in the tuple has been picked to simplify the input and reading of relationships. 

As such, the first entry of the `relations` list below can be read as "the template numbered `0` is positioned `ABOVE_OF` the template numbered `2`".

In [4]:
from tqec.enums import ABOVE_OF, BELOW_OF, LEFT_OF, RIGHT_OF, CornerPositionEnum

relations = [
    (0, ABOVE_OF, 2),
    (1, LEFT_OF, 2),
    (3, RIGHT_OF, 2),
    (4, BELOW_OF, 1),
    (5, BELOW_OF, 2),
    (9, BELOW_OF, 5),
    (8, LEFT_OF, 9),
    (10, RIGHT_OF, 9),
    (11, RIGHT_OF, 10),
    (12, RIGHT_OF, 11),
    (7, ABOVE_OF, 11),
    (13, BELOW_OF, 8),
    (14, BELOW_OF, 9),
    (15, BELOW_OF, 10),
    (16, BELOW_OF, 11),
    (19, BELOW_OF, 14),
    (18, LEFT_OF, 19),
    (20, RIGHT_OF, 19),
    (21, RIGHT_OF, 20),
    (22, RIGHT_OF, 21),
    (17, ABOVE_OF, 22),
    (23, BELOW_OF, 19),
    (24, RIGHT_OF, 23),
    (25, BELOW_OF, 21),
]

### 2.2. Corner pinning

Relative positioning becomes ambiguous when the dimensions of the two "glued" sides do not match. Most of the template share at least one dimension with their neighbours (for example, the template numbered 5 in the image does not share a dimension with 6, but it does share one with 2), but not all of them (look at template numbered 7 in the picture).

To circumvent that, a second way of specifying relations has been added: corner pinning. This method consists in defining a corner on each of the two templates part of the defined relation, and assume that these two corners are stuck together.

*Note*: the `ComposedTemplate` internally uses the corner pinning method and translates the relations using "relative positioning" to corner pinning. Relative positioning is kept in the interface as another, potentially simpler to read and understand, way of encoding relationships between templates.

In [5]:
corner_relations = [
    ((6, CornerPositionEnum.LOWER_LEFT), (CornerPositionEnum.UPPER_RIGHT, 9)),
]

## 3. Construct the template orchestrator

Now that templates and their relative relationships have been defined, creating the `ComposedTemplate` instance is trivial.

In [6]:
from tqec.templates.composed import ComposedTemplate


doubling_template = ComposedTemplate(templates)
for start, reldir, end in relations:
    doubling_template.add_relation(start, reldir, end)

for (start, start_corner), (end_corner, end) in corner_relations:
    doubling_template.add_corner_relation((start, start_corner), (end, end_corner))

## 4. Have fun

The `ComposedTemplate` instance is now created, you can scale it, display it, just have fun playing with it.

In [7]:
from tqec.templates.display import display_template, display_templates_ascii

# Compact visualization:
if False:
    display_template(doubling_template)
# Complete visualization including the separation of the various templates:
else:
    display_templates_ascii(doubling_template)

     ■-------------------■                                  
  .  | .    1    .    1  | .    .    .    .    .    .    .  
■----■-------------------■----■                             
| 1  | 1    1    1    1  | .  | .    .    .    .    .    .  
|    |                   |    |                             
| .  | 1    1    1    1  | 1  | .    .    .    .    .    .  
|    |                   |    |                             
| 1  | 1    1    1    1  | .  | .    .    .    .    .    .  
|    |                   |    |                             
| .  | 1    1    1    1  | 1  | .    .    .    .    .    .  
■----■-------------------■----■                             
| 1  | 1    1    1    1  | .    .    .    .    .    .    .  
■----■                   ■----■    ■-------------------■    
  .  | 1    1    1    1  | 1  | .  | 1    .    1    .  | .  
■----■-------------------■----■----■-------------------■----■
| 1  | 1    1    1    1  | 1    1  | 1    1    1    1  | .  |
|    |                

In [8]:
###############################################################
# WIP: ASCII visualization with boundaries between templates
###############################################################

from tqec.templates.base import Template
from tqec.position import Position
from typing import Tuple, Dict

def new_instantiate(
    sself: Template, plaquette_indices: ty.Sequence[int] 
) -> Tuple[numpy.ndarray, Dict, Dict]:
    """Adapted from ComposedTempalte.instantiate()
    
    Implementation adapted from CompositeTemplate._build_array()
    
    Args:
        sself: the Template instance to instantiate.
        plaquette_indices: the plaquette indices that will be used to
            instantiate the different orchestrated templates.
    """
    indices_map = (0, *plaquette_indices)
    # ul: upper-left
    ul_positions = sself._compute_ul_absolute_position()
    # bbul: bounding-box upper-left
    # bbbr: bounding-box bottom-right
    bbul, bbbr = sself._get_bounding_box_from_ul_positions(ul_positions)
    print(f'bbul={bbul} , bbbr={bbbr}') # <-------------------------------
    shape = sself._get_shape_from_bounding_box(bbul, bbbr)

    ret = numpy.zeros(shape.to_numpy_shape(), dtype=int)
    # Upper-left and bottom-right corners of every template, expressed as (row, column) of the array returned.
    dict_ul = {} 
    dict_br = {}
    print(type(ul_positions))
    # tid: template id
    # tul: template upper-left
    for tid, tul in ul_positions.items():
        #print(f'tid={tid} , tul={tul}') # <-------------------------------
        template = sself._templates[tid]
        # tshapex: template shape x coordinate
        # tshapey: template shape y coordinate
        tshapey, tshapex = template.shape.to_numpy_shape()
        plaquette_indices: list[int] = [
            indices_map[i]
            for i in sself._relative_position_graph.nodes[tid]["plaquette_indices"]
        ]
        # Subtracting bbul (upper-left bounding box position) from each coordinate to stick
        # the represented code to the axes and avoid having negative indices.
        x = tul.x - bbul.x
        y = tul.y - bbul.y
        dict_ul[tid] = (y, x)
        dict_br[tid] = (y+tshapey, x+tshapex)
        # Numpy indexing is (y, x) in our coordinate system convention.
        ret[y : y + tshapey, x : x + tshapex] = template.instantiate(
            plaquette_indices
        )
    return ret, dict_ul, dict_br


def new_display_template(
    template: Template, style: str = 'compact'
) -> None:
    """Adapted from method CompositeTemplate.display_template()

    Args:
        template: the Template instance to display.
        plaquette_indices: the plaquette indices that are forwarded to the call
            to `template.instantiate` to get the actual template representation.
            If None, default to ``range(1, template.expected_plaquettes_number + 1)``.
    """
    assert style in ['compact', 'geometric']
    plaquette_indices = tuple(range(1, template.expected_plaquettes_number + 1))
    print(plaquette_indices)
    #arr = template.instantiate(plaquette_indices)
    arr, ul_pos, br_pos = new_instantiate(template, plaquette_indices)
    print(ul_pos)  # <---------------------------------------------
    print(br_pos)  # <---------------------------------------------
    #print(type(arr))
    #print(numpy.shape(arr))
    h_space = 3 if style == 'compact' else 5
    empty_line = ' '*numpy.shape(arr)[1]*h_space
    # Instead of print to screen, we create a multi-line buffer string.
    # when completed, we print the buffer.
    buffer_lines = []
    if style == 'geometric':
        buffer_lines.append(empty_line)
    for line in arr:
        buffer = ''
        for element in line:
            element = str(element) if element != 0 else "."
            buffer += f"{element:^{h_space}}"
        buffer_lines.append(buffer)
        if style == 'geometric':
            buffer_lines.append(empty_line)
    # Add horizontal and vertical lines.
    if style == 'geometric':
        for tid, tul in ul_pos.items():
            col = tul[1] * h_space
            num_h_sep = br_pos[tid][1]*h_space - col
            # Add horizontal lines at the top and bottom.
            for row in [tul[0], br_pos[tid][0]]:
            #for row in [tul[0]]:
            #for row in [br_pos[tid][0]]:
                row *= 2
                if row == 2:
                    print(f'tid={tid}, row={row}, col={col}, num_h_sep={num_h_sep}')
                line = buffer_lines[row]
                line = line[:col] + '■' + '-'*(num_h_sep-1) + '■' + line[col+num_h_sep+1:]
                buffer_lines[row] = line
            # Add vertical lines on the left and right.
            for col in [tul[1], br_pos[tid][1]]:
                col *= h_space
                for row in range(tul[0]*2+1, br_pos[tid][0]*2):
                    line = buffer_lines[row]
                    line = line[:col] + '|' + line[col+1:]
                    buffer_lines[row] = line
    for line in buffer_lines:
        print(line)


doubling_template.scale_to(2)
new_display_template(doubling_template, style = 'geometric')

(1,)
bbul=Position(x=-1, y=0) , bbbr=Position(x=11, y=18)
<class 'dict'>
{0: (0, 1), 2: (1, 1), 1: (1, 0), 3: (1, 5), 5: (5, 1), 4: (5, 0), 9: (7, 1), 8: (7, 0), 10: (7, 5), 14: (11, 1), 6: (6, 5), 13: (11, 0), 11: (7, 7), 15: (11, 5), 19: (13, 1), 12: (7, 11), 7: (6, 7), 16: (11, 7), 18: (13, 0), 20: (13, 5), 23: (17, 1), 21: (13, 7), 24: (17, 5), 22: (13, 11), 25: (17, 7), 17: (12, 11)}
{0: (1, 5), 2: (5, 5), 1: (5, 1), 3: (5, 6), 5: (7, 5), 4: (6, 1), 9: (11, 5), 8: (11, 1), 10: (11, 7), 14: (13, 5), 6: (7, 6), 13: (12, 1), 11: (11, 11), 15: (13, 7), 19: (17, 5), 12: (11, 12), 7: (7, 11), 16: (13, 11), 18: (17, 1), 20: (17, 7), 23: (18, 5), 21: (17, 11), 24: (18, 6), 22: (17, 12), 25: (18, 11), 17: (13, 12)}
tid=0, row=2, col=5, num_h_sep=20
tid=2, row=2, col=5, num_h_sep=20
tid=1, row=2, col=0, num_h_sep=5
tid=3, row=2, col=25, num_h_sep=5
     ■-------------------■                                  
  .  | .    1    .    1  | .    .    .    .    .    .    .  
■----■----------------

In [9]:
display_template(doubling_template.scale_to(3))

  .  .  1  .  1  .  1  .  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .
  1  1  1  1  1  1  1  .  .  .  .  .  .  .  .  .
  .  1  1  1  1  1  1  1  .  1  .  1  .  1  .  .
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  .
  .  1  1  1  1  1  

In [10]:
plaquette_indices = list(range(1, doubling_template.expected_plaquettes_number + 1))
arr = doubling_template.scale_to(500).instantiate(plaquette_indices)
print("Array shape:", arr.shape)

Array shape: (3006, 2004)
